In [2]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig
from trl import SFTTrainer
from tqdm import tqdm

df = pd.read_csv("edos_labelled_aggregated.csv")
df = df[df['label_category'] != 'none']

train_df, dev_df, test_df = df[df['split'] == 'train'], df[df['split'] == 'dev'], df[df['split'] == 'test']

print(f"train: {train_df.shape[0]}, dev:{dev_df.shape[0]}, test:{test_df.shape[0]}")

# Define the prompt template
prompt_template = """Category of Sexism: for posts which are sexist, a four-class classification where systems have to predict one of four categories: (1) threats, (2)  derogation, (3) animosity, (4) prejudiced discussion. 

Given a post determine the post is belong to which class:
1. threats, plans to harm and incitement
2. derogation 
3. animosity
4. prejudiced discussions

### Post: {POST}
### Answer: """

column='label_category'

train: 3398, dev:486, test:970


In [3]:
llm_path = "task_b_llm"

tokenizer = AutoTokenizer.from_pretrained(llm_path, padding_side="left")

tokenizer.pad_token = tokenizer.eos_token

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model with 4-bit precision
finetuned_model = AutoModelForCausalLM.from_pretrained(llm_path, quantization_config=quant_config, device_map={"": 0})

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
# len(tokenizer.encode("1. threats, plans to harm and incitement"))

In [5]:
from torch.utils.data import DataLoader

class EDOSDataset(Dataset):
    def __init__(self, df, prompt_template, column):
        self.texts = df['text'].tolist()
        self.labels = df[column].tolist()
        self.prompt_template=prompt_template

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idxs):
        inputs, inputs_labels = [], []
        for idx in idxs:
            
            inputs.append(self.prompt_template.replace("{POST}", self.texts[idx]))
            inputs_labels.append(self.labels[idx])
        
        return {"inputs":inputs, "labels": inputs_labels}
    
def make_the_generations(model, tokenizer, data_loader):
    gen_texts, labels = [], []
    
    for batch in tqdm(data_loader):
        input_data = batch['inputs']
        labels += batch['labels']
        tokenized_input_data = tokenizer(input_data, padding=True, max_length=512, truncation=True, return_tensors="pt").to("cuda:0")
        # print(tokenized_input_data)
        outputs = finetuned_model.generate(
            **tokenized_input_data,
            pad_token_id= tokenizer.eos_token_id,
            max_new_tokens=15,
            do_sample=False
        )
        generated_texts = [tokenizer.decode(outputs[idx], skip_special_tokens=True)[len(input_data[idx]):]
                          for idx in range(len(outputs))]
        gen_texts += generated_texts
    return gen_texts, labels

In [6]:
batch_size = 64

train_data = EDOSDataset(df=train_df, prompt_template=prompt_template, column=column)
train_dataloader =  DataLoader(train_data, batch_size=batch_size, shuffle=False)
train_texts, train_labels = make_the_generations(finetuned_model, tokenizer, train_dataloader)

100%|██████████| 54/54 [01:22<00:00,  1.52s/it]


In [7]:
dev_data = EDOSDataset(df=dev_df, prompt_template=prompt_template, column=column)
dev_dataloader =  DataLoader(dev_data, batch_size=batch_size, shuffle=False)
dev_texts, dev_labels = make_the_generations(finetuned_model, tokenizer, dev_dataloader)

100%|██████████| 8/8 [00:11<00:00,  1.48s/it]


In [8]:
test_data = EDOSDataset(df=test_df, prompt_template=prompt_template, column=column)
test_dataloader =  DataLoader(test_data, batch_size=batch_size, shuffle=False)
test_texts, test_labels = make_the_generations(finetuned_model, tokenizer, test_dataloader)

100%|██████████| 16/16 [00:23<00:00,  1.49s/it]


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

vectorizer = TfidfVectorizer(ngram_range=(1,3), lowercase=True)

class_mapper = Pipeline (
    steps=[
        ("Vectorizer", vectorizer), 
        ('Classifier', LogisticRegression())
])

class_mapper.fit(train_texts+train_labels, train_labels+train_labels)
# +train_texts

Pipeline(steps=[('Vectorizer', TfidfVectorizer(ngram_range=(1, 3))),
                ('Classifier', LogisticRegression())])

In [10]:
train_predict = class_mapper.predict(train_texts)
dev_predict = class_mapper.predict(dev_texts)
test_predict = class_mapper.predict(test_texts)

print("TRAIN"+"-"*150)
print(classification_report(train_labels, train_predict, digits=4))
print("DEV"+"-"*150)
print(classification_report(dev_labels, dev_predict, digits=4))
print("TEST"+"-"*150)
print(classification_report(test_labels, test_predict, digits=4))

TRAIN------------------------------------------------------------------------------------------------------------------------------------------------------
                                          precision    recall  f1-score   support

1. threats, plans to harm and incitement     0.9854    0.8710    0.9247       310
                           2. derogation     0.9349    0.9579    0.9463      1590
                            3. animosity     0.9414    0.9382    0.9398      1165
               4. prejudiced discussions     0.8862    0.8889    0.8876       333

                                accuracy                         0.9364      3398
                               macro avg     0.9370    0.9140    0.9246      3398
                            weighted avg     0.9370    0.9364    0.9363      3398

DEV------------------------------------------------------------------------------------------------------------------------------------------------------
                               

In [4]:
3398*3

10194

In [2]:
1199*24

28776